In [2]:
# bibliotecas
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.preprocessing import OneHotEncoder, StandardScaler, PolynomialFeatures
from sklearn.metrics import mean_absolute_error, mean_squared_error

from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
import lightgbm as lgb
from catboost.core import CatBoostRegressor

In [3]:
# dados
df = pd.read_csv('../data/insurance-encoded.csv', sep=',', encoding='utf-8')
df.head()

,age,sex,bmi,children,smoker,region,charges
0,19,0,27.900,0,1,3,16884.92400
1,18,1,33.770,1,0,2,1725.55230
2,28,1,33.000,3,0,2,4449.46200
3,33,1,22.705,0,0,1,21984.47061
4,32,1,28.880,0,0,1,3866.85520


In [13]:
# variaveis
x = df.drop('charges', axis=1)
y = df.charges

# base treino x teste
x_treino, x_teste, y_treino, y_teste = train_test_split(x,y,test_size=0.3, random_state=0)

metricas = {}

In [5]:
# shapes
x_treino.shape, x_teste.shape, y_treino.shape, y_teste.shape


((936, 6), (402, 6), (936,), (402,))

## linear regression

In [14]:
# modelo
nome_modelo = 'Linear regression'
lr = LinearRegression().fit(x_treino, y_treino)

# metricas
y_prev = lr.predict(x_teste)
mae = mean_absolute_error(y_teste, y_prev)
mse = mean_squared_error(y_teste, y_prev)
rmse = np.sqrt(mse)
r2_treino = lr.score(x_treino, y_treino)
r2_teste = lr.score(x_teste, y_teste)
cv = cross_val_score(lr, x, y, cv=KFold(n_splits=15,shuffle=True,random_state=5)).mean()

# resultados
print('MAE: %.3f' % mae)
print('MSE: %.3f' % mse)
print('RMSE: %.3f' % rmse)
print('R2 TREINO: %.2f%%' % (r2_treino*100))
print('R2 TESTE: %.2f%%' % (r2_teste*100))
print('VALIDAÇÃO CRUZADA: %.2f%%' % (cv*100))

# persistencia
metricas[nome_modelo] = {
    'MAE':mae, 'MSE':mse, 'RMSE':rmse, 'R2_treino':r2_treino, 'R2_teste':r2_teste, 'Cross_val':cv
}

MAE: 4013.693
MSE: 33311355.282
RMSE: 5771.599
R2 TREINO: 73.07%
R2 TESTE: 79.11%
VALIDAÇÃO CRUZADA: 73.60%


## polinomial

In [15]:
# pre processamento
pf = PolynomialFeatures(degree=2)
x_poly = pf.fit_transform(x)
x_poly_treino = pf.fit_transform(x_treino)
x_poly_teste = pf.fit_transform(x_teste)

# modelo
plr = LinearRegression().fit(x_poly_treino, y_treino)
nome_modelo = 'Polynomial regression'

# metricas
y_prev = plr.predict(x_poly_teste)
mae = mean_absolute_error(y_teste, y_prev)
mse = mean_squared_error(y_teste, y_prev)
rmse = np.sqrt(mse)
r2_treino = plr.score(x_poly_treino, y_treino)
r2_teste = plr.score(x_poly_teste, y_teste)
cv = cross_val_score(plr, x_poly, y, cv=KFold(n_splits=15,shuffle=True,random_state=5)).mean()

# resultados
print('MAE: %.3f' % mae)
print('MSE: %.3f' % mse)
print('RMSE: %.3f' % rmse)
print('R2 TREINO: %.2f%%' % (r2_treino*100))
print('R2 TESTE: %.2f%%' % (r2_teste*100))
print('VALIDAÇÃO CRUZADA: %.2f%%' % (cv*100))

# persistencia
metricas[nome_modelo] = {
    'MAE':mae, 'MSE':mse, 'RMSE':rmse, 'R2_treino':r2_treino, 'R2_teste':r2_teste, 'Cross_val':cv
}

MAE: 2855.525
MSE: 20028939.399
RMSE: 4475.370
R2 TREINO: 83.12%
R2 TESTE: 87.44%
VALIDAÇÃO CRUZADA: 77.57%


## SVR

In [17]:
# pre processamento
scaler = StandardScaler()
x_esc = scaler.fit_transform(x)
y_esc = scaler.fit_transform(y.values.reshape(-1,1))

x_treino_esc, x_teste_esc, y_treino_esc, y_teste_esc = train_test_split(x_esc, y_esc, test_size=0.3, random_state=0)

# modelo
svr = SVR().fit(x_treino_esc, y_treino_esc.ravel())
nome_modelo = 'Support vector machine'
y_prev = svr.predict(x_teste_esc)
y_teste_inverse = scaler.inverse_transform(y_teste_esc)
y_prev_inverse = scaler.inverse_transform(y_prev.reshape(-1, 1))


# metricas
mae = mean_absolute_error(y_teste_inverse, y_prev_inverse)
mse = mean_squared_error(y_teste_inverse, y_prev_inverse)
rmse = np.sqrt(mse)
r2_treino = svr.score(x_treino_esc, y_treino_esc)
r2_teste = svr.score(x_teste_esc, y_teste_esc)
cv = cross_val_score(svr, x_esc, y_esc.ravel(), cv=KFold(n_splits=15,shuffle=True,random_state=5)).mean()

# resultados
print('MAE: %.3f' % mae)
print('MSE: %.3f' % mse)
print('RMSE: %.3f' % rmse)
print('R2 TREINO: %.2f%%' % (r2_treino*100))
print('R2 TESTE: %.2f%%' % (r2_teste*100))
print('VALIDAÇÃO CRUZADA: %.2f%%' % (cv*100))

# persistencia
metricas[nome_modelo] = {
    'MAE':mae, 'MSE':mse, 'RMSE':rmse, 'R2_treino':r2_treino, 'R2_teste':r2_teste, 'Cross_val':cv
}

MAE: 2458.986
MSE: 19401893.161
RMSE: 4404.758
R2 TREINO: 84.26%
R2 TESTE: 87.83%
VALIDAÇÃO CRUZADA: 83.50%


## decision tree

In [18]:
# modelo
tree = DecisionTreeRegressor(random_state=10, max_depth=4).fit(x_treino, y_treino)
nome_modelo = 'Decision tree'

# metricas
y_prev = tree.predict(x_teste)
mae = mean_absolute_error(y_teste, y_prev)
mse = mean_squared_error(y_teste, y_prev)
rmse = np.sqrt(mse)
r2_treino = tree.score(x_treino, y_treino)
r2_teste = tree.score(x_teste, y_teste)
cv = cross_val_score(tree, x, y, cv=KFold(n_splits=15,shuffle=True,random_state=0)).mean()

# resultados
print('MAE: %.3f' % mae)
print('MSE: %.3f' % mse)
print('RMSE: %.3f' % rmse)
print('R2 TREINO: %.2f%%' % (r2_treino*100))
print('R2 TESTE: %.2f%%' % (r2_teste*100))
print('VALIDAÇÃO CRUZADA: %.2f%%' % (cv*100))

# persistencia
metricas[nome_modelo] = {
    'MAE':mae, 'MSE':mse, 'RMSE':rmse, 'R2_treino':r2_treino, 'R2_teste':r2_teste, 'Cross_val':cv
}

MAE: 2595.096
MSE: 18733722.545
RMSE: 4328.247
R2 TREINO: 85.30%
R2 TESTE: 88.25%
VALIDAÇÃO CRUZADA: 84.71%


## random forest

In [19]:
# modelo
rf = RandomForestRegressor(random_state=10, criterion='squared_error', max_depth=4, n_estimators=20).fit(x_treino, y_treino)
nome_modelo = 'Random forest'

# metricas
y_prev = rf.predict(x_teste)
mae = mean_absolute_error(y_teste, y_prev)
mse = mean_squared_error(y_teste, y_prev)
rmse = np.sqrt(mse)
r2_treino = rf.score(x_treino, y_treino)
r2_teste = rf.score(x_teste, y_teste)
cv = cross_val_score(rf, x, y, cv=KFold(n_splits=15,shuffle=True,random_state=0)).mean()

# resultados
print('MAE: %.3f' % mae)
print('MSE: %.3f' % mse)
print('RMSE: %.3f' % rmse)
print('R2 TREINO: %.2f%%' % (r2_treino*100))
print('R2 TESTE: %.2f%%' % (r2_teste*100))
print('VALIDAÇÃO CRUZADA: %.2f%%' % (cv*100))

# persistencia
metricas[nome_modelo] = {
    'MAE':mae, 'MSE':mse, 'RMSE':rmse, 'R2_treino':r2_treino, 'R2_teste':r2_teste, 'Cross_val':cv
}

MAE: 2511.010
MSE: 17300470.050
RMSE: 4159.383
R2 TREINO: 86.39%
R2 TESTE: 89.15%
VALIDAÇÃO CRUZADA: 85.45%


## xgboost

In [20]:
# modelo
xgb = XGBRegressor(random_state=10, n_estimators=20, max_depth=2).fit(x_treino, y_treino)
nome_modelo = 'XGBoost'

# metricas
y_prev = xgb.predict(x_teste)
mae = mean_absolute_error(y_teste, y_prev)
mse = mean_squared_error(y_teste, y_prev)
rmse = np.sqrt(mse)
r2_treino = xgb.score(x_treino, y_treino)
r2_teste = xgb.score(x_teste, y_teste)
cv = cross_val_score(xgb, x, y, cv=KFold(n_splits=15,shuffle=True,random_state=0)).mean()

# resultados
print('MAE: %.3f' % mae)
print('MSE: %.3f' % mse)
print('RMSE: %.3f' % rmse)
print('R2 TREINO: %.2f%%' % (r2_treino*100))
print('R2 TESTE: %.2f%%' % (r2_teste*100))
print('VALIDAÇÃO CRUZADA: %.2f%%' % (cv*100))

# persistencia
metricas[nome_modelo] = {
    'MAE':mae, 'MSE':mse, 'RMSE':rmse, 'R2_treino':r2_treino, 'R2_teste':r2_teste, 'Cross_val':cv
}

MAE: 2466.208
MSE: 16932351.599
RMSE: 4114.894
R2 TREINO: 86.11%
R2 TESTE: 89.38%
VALIDAÇÃO CRUZADA: 85.61%


## lightgbm

In [21]:
# modelo
lgbm = lgb.LGBMRegressor(random_state=10,num_leaves=4, max_depth=2, n_estimators=70, verbose=-1).fit(x_treino, y_treino)
nome_modelo = 'LightGBM'

# metricas
y_prev = lgbm.predict(x_teste)
mae = mean_absolute_error(y_teste, y_prev)
mse = mean_squared_error(y_teste, y_prev)
rmse = np.sqrt(mse)
r2_treino = lgbm.score(x_treino, y_treino)
r2_teste = lgbm.score(x_teste, y_teste)
cv = cross_val_score(lgbm, x, y, cv=KFold(n_splits=15,shuffle=True,random_state=0)).mean()

# resultados
print('MAE: %.3f' % mae)
print('MSE: %.3f' % mse)
print('RMSE: %.3f' % rmse)
print('R2 TREINO: %.2f%%' % (r2_treino*100))
print('R2 TESTE: %.2f%%' % (r2_teste*100))
print('VALIDAÇÃO CRUZADA: %.2f%%' % (cv*100))

# persistencia
metricas[nome_modelo] = {
    'MAE':mae, 'MSE':mse, 'RMSE':rmse, 'R2_treino':r2_treino, 'R2_teste':r2_teste, 'Cross_val':cv
}

MAE: 2460.351
MSE: 16997371.660
RMSE: 4122.787
R2 TREINO: 85.98%
R2 TESTE: 89.34%
VALIDAÇÃO CRUZADA: 85.60%


## catboost

In [22]:
# modelo
cat = CatBoostRegressor(random_state=10, depth=2, iterations=100, verbose=0).fit(x_treino, y_treino)
nome_modelo = 'Catboost'

# metricas
y_prev = cat.predict(x_teste)
mae = mean_absolute_error(y_teste, y_prev)
mse = mean_squared_error(y_teste, y_prev)
rmse = np.sqrt(mse)
r2_treino = cat.score(x_treino, y_treino)
r2_teste = cat.score(x_teste, y_teste)
cv = cross_val_score(cat, x, y, cv=KFold(n_splits=15,shuffle=True,random_state=0)).mean()

# resultados
print('MAE: %.3f' % mae)
print('MSE: %.3f' % mse)
print('RMSE: %.3f' % rmse)
print('R2 TREINO: %.2f%%' % (r2_treino*100))
print('R2 TESTE: %.2f%%' % (r2_teste*100))
print('VALIDAÇÃO CRUZADA: %.2f%%' % (cv*100))

# persistencia
metricas[nome_modelo] = {
    'MAE':mae, 'MSE':mse, 'RMSE':rmse, 'R2_treino':r2_treino, 'R2_teste':r2_teste, 'Cross_val':cv
}

MAE: 2449.538
MSE: 17239741.971
RMSE: 4152.077
R2 TREINO: 86.96%
R2 TESTE: 89.19%
VALIDAÇÃO CRUZADA: 85.60%


In [35]:
pd.DataFrame(metricas).T.sort_values('R2_teste', ascending=False)

,MAE,MSE,RMSE,R2_treino,R2_teste,Cross_val
XGBoost,2466.207921,1.693235e+07,4114.893875,0.861067,0.893821,0.856133
LightGBM,2460.350987,1.699737e+07,4122.786880,0.859830,0.893413,0.856026
Catboost,2449.537856,1.723974e+07,4152.076826,0.869618,0.891893,0.855981
Random forest,2511.010058,1.730047e+07,4159.383374,0.863906,0.891512,0.854473
Decision tree,2595.096071,1.873372e+07,4328.247052,0.853007,0.882525,0.847103
support vector machine,2458.986013,1.940189e+07,4404.758014,0.842574,0.878335,0.834984
Polynomial regression,2855.524563,2.002894e+07,4475.370309,0.831184,0.874403,0.775705
Linear regression,4013.692986,3.331136e+07,5771.599023,0.730684,0.791111,0.735995
